<a href="https://colab.research.google.com/github/jaalvalcan/GEE_index_sets/blob/main/Moondream_model__to__detect__objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages

Uncomment the following line to install the required packages.

In [ ]:
%pip install -U geoai-py

In [ ]:
!pip install triton

## Import libraries

In [ ]:
import leafmap
from geoai import MoondreamGeo
import geoai

## Download sample data

We'll use a sample GeoTIFF image of a parking lot with buildings and vegetation.

In [ ]:
url = "/content/TURK1_clipped.tif"
image_path = geoai.download_file(url)
image_path

## Visualize the image

Let's first visualize the sample image on an interactive map.

In [ ]:
m = leafmap.Map()
m.add_raster(image_path, layer_name="Satellite Image")
m

## Initialize the Moondream processor

Load the Moondream2 model. The first time you run this, the model will be downloaded from HuggingFace (~3.7GB).

**Note**: For reproducibility, we specify a specific model revision date.

In [ ]:
processor = MoondreamGeo(
    model_name="vikhyatk/moondream2", revision="2025-06-21", device="cpu"
)

## Image Captioning

Generate a description of the satellite image. The `length` parameter controls the detail level: "short", "normal", or "long".

In [ ]:
result = processor.caption(image_path, length="normal")
print(result["caption"])

## Visual Question Answering

Ask questions about the image content and get natural language answers.

In [ ]:
result = processor.query("How many building are in the image?", image_path)
print(result["answer"])

In [ ]:
result = processor.query("What are the building roof colors?", image_path)
print(result["answer"])

## Object Detection

Detect objects and get bounding boxes. Results are automatically georeferenced when using GeoTIFF input.

### Detect buildings

In [ ]:
result = processor.detect(image_path, "building", output_path="buildings.geojson")
print(f"Detected {len(result['objects'])} buildings")

View the GeoDataFrame with georeferenced bounding boxes:

In [ ]:
result["gdf"]

Add the detected buildings to the map:

In [ ]:
m.add_gdf(result["gdf"], layer_name="buildings")
m

## Point Localization

Find object center points instead of bounding boxes. Useful for counting and locating objects.

### Find building centroids

In [ ]:
result = processor.point(
    image_path, "building", output_path="building_centroids.geojson"
)
print(f"Found {len(result['points'])} building centroids")

In [ ]:
m.add_gdf(result["gdf"], layer_name="building_centroids")

### Detect trees

In [ ]:
result = processor.detect(image_path, "tree", output_path="trees.geojson")
print(f"Detected {len(result['objects'])} trees")

In [ ]:
m.add_gdf(result["gdf"], layer_name="trees")

### Find tree centroids

In [ ]:
result = processor.point(image_path, "trees", output_path="tree_centroids.geojson")
print(f"Found {len(result['points'])} tree centroids")

In [ ]:
m.add_gdf(result["gdf"], layer_name="tree_centroids")

## Display final map

View all detected objects and centroids on the map.


In [ ]:
m

![](https://github.com/user-attachments/assets/032daf91-1cb3-464b-9576-17f61767ab6f)